<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/LSTM_Keras_embeddings_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
#USING Keras embeddings and LSTM deep learning model: 
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [4]:
train.head(1)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1


In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train2 = train.drop(columns=['description', 'id'])

In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [8]:
train3.head(1)

,label,text
0,1,"Sometimes, when whisky is batched, a few lefto..."


In [44]:
train3['label'].unique()

array([1, 0, 2])

In [9]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.20, random_state=42)

df_trn.shape, df_val.shape

((3269, 2), (818, 2))

In [0]:
#Define train and val datasets and X, Y vectors

In [0]:
X_train = df_trn['text'].values
y_train = df_trn['label'].values
X_val = df_val['text'].values
y_val = df_val['label'].values


In [0]:
#Tokenize using Keras 

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer_obj = Tokenizer()
total_reviews = df_trn['text'].values
tokenizer_obj.fit_on_texts(total_reviews)

In [0]:
#pad sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
max_length = max([len(s.split()) for s in total_reviews])

In [0]:
#define vocabulary size

vocab_size = len(tokenizer_obj.word_index) + 1

In [0]:
#define X_train and X_val tokens

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)

X_val_tokens = tokenizer_obj.texts_to_sequences(X_val)


In [0]:
#Define X_train and X_val pads

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding='post')

X_val_pad = pad_sequences(X_val_tokens, maxlen = max_length, padding='post')

In [28]:
X_train_pad

array([[   46,   276,     5, ...,     0,     0,     0],
       [    2,   121,  4203, ...,     0,     0,     0],
       [    3,    16,    14, ...,     0,     0,     0],
       ...,
       [    2, 13222,     4, ...,     0,     0,     0],
       [ 2193,  3969,    28, ...,     0,     0,     0],
       [  218,     6,  1048, ...,     0,     0,     0]], dtype=int32)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_pad)

X_val_scaled = scaler.transform(X_val_pad)


In [0]:
#Build Keras Model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Activation, SpatialDropout1D
from tensorflow.keras.optimizers import Adam

In [0]:
#The Embedding layer requires the specification of the vocabulary size (vocab_size), the size of the real-valued vector space EMBEDDING_DIM = 100, and the maximum length of input documents max_length

In [0]:
#https://towardsdatascience.com/17-rules-of-thumb-for-building-a-neural-network-93356f9930af

model = Sequential()

In [0]:
#stacking layers on model


In [96]:
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [97]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 237, 100)          1322800   
_________________________________________________________________
spatial_dropout1d_18 (Spatia (None, 237, 100)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_20 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 65        
Total params: 1,409,729
Trainable params: 1,409,729
Non-trainable params: 0
___________________________________________

In [98]:
model.fit(X_train_scaled, y_train, batch_size=30, epochs=5, verbose=2, validation_data=(X_val_scaled, y_val))

#start with 100 epochs then tune after viewing the losses and accuracy plot
#increasing batch size may increase accuracy

Epoch 1/5
109/109 - 129s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 2/5
109/109 - 128s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 3/5
109/109 - 126s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 4/5
109/109 - 127s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 5/5
109/109 - 127s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
